In [1]:
import os
import analysis
import hrf_tools
import numpy as np
from utility import mask_cifti,unmask_cifti
from analysis import get_vertex_info_59k,simple_ridgeCV,plot_results
import hrf_tools
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
#import matplotlib
#matplotlib.rcParams['figure.figsize'] = (6, 4)
# this is run by run_datalad_run_encoding_model.sh
# "python ./run_encoding_model.py $sub $2"

/opt/miniconda-latest/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
## plan
# 1: extract auditory areas (?)

#no... these are just the surface and don't match up exactly with the cifti

# run model on just specific features and combine across movies?
# plot results of specific features


# 2 extract primary auditory cortex
# get mfs features? (coarse)
# try to plot tonotopy

## get EARLY AC and ALL AC from all subjects - store eachon scratch

In [ ]:
subfile = open("HCP_7T_subjects.txt", "r")
sublist = [(line.strip()).split() for line in subfile]
sublist = [item for s in sublist for item in s]

feature='mfs'
dataset='HCP_7T'
n_movies=[1,2,3,4]
#X_all=[]
#X_all_4=[]
Y_all=[]
Y_all_early=[]
subject_list=[]

for i,subject in enumerate(sublist):
    print(f'{i+1} of {len(sublist)}')
    
    try:
        X,Y,vertex_info = analysis.load_data_HCP(subject,feature,n_movies)
        #X_all.append(X)
        #X_new=np.zeros((X.shape[0],4))
#         for i in np.arange(4):
#             start=i*32
#             stop=(i*32)+32
#             X_new[:,i]=np.mean(X[:,start:stop],axis=1)
        Y_AC=mask_cifti(auditory_59k,Y)
        Y_AC_early=mask_cifti(early_auditory_59k,Y)

#         X_all_4.append(X_new)
#         Y_all.append(Y_new)
#         Y_all_early.append(Y_new_early)
        subject_list.append(subject)
    except:
        print(f'file dne, skipping {subject}')
        continue
# np.save('/om/scratch/Thu/jsmentch/X_all.npy',X_all) #this is all mfs features
# np.save('/om/scratch/Thu/jsmentch/X_all_4.npy',X_all_4) #this is the same features but combined into 4 bins

    np.save(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/AC/{subject}.npy',Y_AC) #this is early and association AC
    np.save(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/AC_early/{subject}.npy',Y_AC_early) #this is early AC
np.save('/om/scratch/Thu/jsmentch/HCP_7T_rois/subject_list.npy',subject_list) #this is the list of subjects that we were able to get data from

#### to convert old list of arrays of mfs features to a single mfs feature because they are the same for all subjects
# X_all=np.load('/om/scratch/Thu/jsmentch/X_all.npy') #this is all mfs features
# X_all_4=np.load('/om/scratch/Thu/jsmentch/X_all_4.npy') #this is the same features but combined into 4 bins
# X_all=X_all[0]
# X_all_4=X_all_4[0]
# print(X_all.shape,X_all_4.shape)
# np.save(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/mfs_all.npy',X_all)
# np.save(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/mfs_4.npy',X_all_4)

In [10]:
#save the other AS features in the same way as mfs above concatenated across movies
features=['chroma', 'mfcc', 'mfs', 'as_embed', 'as_scores']
subject=100610
dataset='HCP_7T'
for feature in features:
    n_movies=[1,2,3,4]
    X,Y,vertex_info = analysis.load_data_HCP(subject,feature,n_movies)

    np.save(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/{feature}.npy',X)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-

## run encoding model - train on all subs except 1, test on 1

In [2]:
mmp_59k=np.load('../sourcedata/data/parcellations/mmp_59k.npy' )
early_auditory_59k=np.load('../sourcedata/data/parcellations/early_auditory_59k.npy' )
auditory_59k=np.load('../sourcedata/data/parcellations/auditory_59k.npy' )

In [3]:
vertex_info=get_vertex_info_59k()

In [11]:
features=['chroma', 'mfcc', 'mfs', 'as_embed', 'as_scores']

subject_list=np.load('/om/scratch/Thu/jsmentch/HCP_7T_rois/subject_list.npy') #this is the list of subjects that we were able to get data from
mfs_all=np.load(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/mfs_all.npy')
mfs_4=np.load(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/mfs_4.npy')
chroma=np.load(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/chroma.npy')
mfcc=np.load(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/mfs_4.npy')
as_embed=np.load(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/as_embed.npy')
as_scores=np.load(f'/om/scratch/Thu/jsmentch/HCP_7T_rois/as_scores.npy')


In [13]:
feature='as_scores'
num_sub=20

#apply HRF and standard scale X feature
scaler = StandardScaler()
X_h= hrf_tools.apply_optimal_hrf_10hz(as_scores,1)
X_s = scaler.fit_transform(X=X_h,y=None)

X=[]
Y=[]
for subject in subject_list[0:num_sub]:
    X.append(X_s)
    Y_=np.load (f'/om/scratch/Thu/jsmentch/HCP_7T_rois/AC/{subject}.npy') 
    Y.append(Y_)
#Y = np.load ('/om/scratch/Thu/jsmentch/Y_all.npy') #this is all AC
Y=np.concatenate(Y)
X=np.concatenate(X)

In [14]:
scores_mean,corr_mean,weights_mean=simple_ridgeCV(X,Y,num_sub)
scores_mean_unmasked=unmask_cifti(auditory_59k,scores_mean)

In [15]:
plot_results(scores_mean_unmasked,'r2','59k',vertex_info,'hcp_test',feature,'HCP_7T',f'{num_sub}_sub_cv{num_sub}_AC_')

Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data


/om2/vast/gablab/jsmentch/projects/nat_img/code/nilearn_plotting_custom.py:184: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  axes = Axes3D(figure, rect=[0, 0, 1, 1],


[-0.01599029 -0.01565468 -0.01546848 ...  0.43341737  0.43373959
  0.43691689]
[-0.0196055  -0.01960466 -0.0193408  ...  0.3793563   0.39278789
  0.39485723]
[-0.01599029 -0.01565468 -0.01546848 ...  0.43341737  0.43373959
  0.43691689]
[-0.0196055  -0.01960466 -0.0193408  ...  0.3793563   0.39278789
  0.39485723]


## run encoding model - train on all subs 3 movies, test on 1 movie